# Generate UT-Zap50K results

Distribution A. Approved for public release: distribution unlimited.  Case 88ABW-2019-1334. 27 March 2019.

THIS SOFTWARE AND ANY ACCOMPANYING DOCUMENTATION IS    
RELEASED "AS IS." THE US GOVERNMENT MAKES NO WARRANTY  
OF ANY KIND, EXPRESS OR IMPLIED, CONCERNING THIS SOFTWARE  
AND ANY ACCOMPANYING DOCUMENTATION, INCLUDING,  
WITHOUT LIMITATION, ANY WARRANTIES OF MERCHANTABILITY   
OR FITNES FOR A PARTICULAR PURPOSE. IN NO EVENT WILL THE  
US GOVERNMENT BE LIABLE FOR ANY DAMAGES, INCLUDING ANY  
LOST PROFITS, LOST SAVINGS, OR OTHER INCIDENTAL OR  
CONSEQUENTIAL DAMAGES ARISING OUT OF THE USE, OR  
INABILITY TO USE, THIS SOFTWARE OR ANY ACCOMPANYING   
DOCUMENTATION, EVEN IF INFORMED IN ADVANCE OF THE  
POSSIBILITY OF SUCH DAMAGES.    

## Setup

In [13]:
##
## Environment
##
import os
from tool import *
from unkunk_utility import *
from clustering import *
from optim import *
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=str(1)
import foolbox
import numpy as np
import keras
import random
import math
import tabulate
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Input, Dense, Activation, Conv2D, MaxPooling2D, Flatten, regularizers, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import HTML, display
from foolbox.criteria import TargetClass
from foolbox.distances import MAE

from scipy.interpolate import interp1d
import statsmodels.api as sm

##
## Model params
##
boundarySteps = 1000

##
## Unknown search params
##  
criticalClass = 0
sampleSize = 2000
toSelect = 50

## Zappos

### Build model

- Make dataset 
- Load data
- Fit model
- Evaluate model

In [3]:
##
## Make dataset (Creates numpy arrays for the zappos datasets)
##
# bootPath = 'ut-zap50k-images-square/Boots/'
# sandalPath = 'ut-zap50k-images-square/Sandals/'
# shoePath = 'ut-zap50k-images-square/Shoes/'
# slipperPath = 'ut-zap50k-images-square/Slippers/'
# ROW = 64
# COL = 64
# zapposDatasetMaker(bootPath, sandalPath, shoePath, slipperPath, ROW, COL)

##
## Load dataset
##
trainImages = np.load('trainImages.npy')
trainLabels = np.load('trainLabels.npy')
valImages = np.load('valImages.npy')
valLabels = np.load('valLabels.npy')
testImages = np.load('testImages.npy')
testLabels = np.load('testLabels.npy')   

##
## Want not-shoe vs shoes (remove boots because they are way too easy to find)
##
trainDel = np.where(trainLabels == 0)    ## Remove easy to find boots
valDel = np.where(valLabels == 0)        ## Remove easy to find boots
testDel = np.where(testLabels == 0)      ## Remove easy to find boots

trainImages = np.delete(trainImages, trainDel, axis = 0)
trainLabels = np.delete(trainLabels, trainDel)

valImages = np.delete(valImages, valDel, axis = 0)
valLabels = np.delete(valLabels, valDel)

testImages = np.delete(testImages, testDel, axis = 0)
testLabels = np.delete(testLabels, testDel)

trainLabels[trainLabels != 2] = 0      ## Recode to zero and one
trainLabels[trainLabels == 2] = 1      ## Recode to zero and one

valLabels[valLabels != 2] = 0          ## Recode to zero and one
valLabels[valLabels == 2] = 1          ## Recode to zero and one

testLabels[testLabels != 2] = 0        ## Recode to zero and one
testLabels[testLabels == 2] = 1        ## Recode to zero and one

##
## Fit model
##
model = modelMaker(64, 64, 3, 2)
model.fit(trainImages,
           keras.utils.to_categorical(trainLabels),
           batch_size = 128,
           epochs=75,
           verbose = 2)
model.save('zappos.h5')
model = load_model('zappos.h5')

##
## Evaluate
##
trainAcc = model.evaluate(trainImages, keras.utils.to_categorical(trainLabels), verbose=0)[1]
predictedTrain = np.argmax(model.predict(trainImages), axis = 1)
confidenceTrain = np.max(model.predict(trainImages), axis = 1)
trainECE = ECE(confidenceTrain, predictedTrain, trainLabels, 15)

valAcc = model.evaluate(valImages, keras.utils.to_categorical(valLabels), verbose=0)[1]
predictedVal = np.argmax(model.predict(valImages), axis = 1)
confidenceVal = np.max(model.predict(valImages), axis = 1)
valECE = ECE(confidenceVal, predictedVal, valLabels, 15)

testAcc = model.evaluate(testImages, keras.utils.to_categorical(testLabels), verbose=0)[1]
predictedTest = np.argmax(model.predict(testImages), axis = 1)
confidenceTest = np.max(model.predict(testImages), axis = 1)
testECE = ECE(confidenceTest, predictedTest, testLabels, 15)

table = [['Measure', 'Acc', 'ECE'],
         ['Train', round(trainAcc, 2), round(trainECE, 2)],
         ['Val', round(valAcc, 2), round(valECE, 2)],
         ['Test', round(testAcc, 2), round(testECE, 2)]]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

reliabilityDiagram(confidenceTest, predictedTest, testLabels, 15)

- This model is not calibrated

### Calibrate model

- Calibrate model
- Evaluate model

In [ ]:
##
## Calibrate
##
logitExtractor = logitExtract(model, 'output')
calibrate = calibrateModel(logitExtractor, valImages, valLabels, classes = 2, epochs = 500)

##
## Extract values
##
trainAcc = model.evaluate(trainImages, keras.utils.to_categorical(trainLabels), verbose=0)[1]
predictedTrain = np.argmax(model.predict(trainImages), axis = 1)
confidenceTrain = np.max(calibrate.predict(logitExtractor.predict(trainImages)), axis = 1)
trainECE = ECE(confidenceTrain, predictedTrain, trainLabels, 15)

valAcc = model.evaluate(valImages, keras.utils.to_categorical(valLabels), verbose=0)[1]
predictedVal = np.argmax(model.predict(valImages), axis = 1)
confidenceVal = np.max(calibrate.predict(logitExtractor.predict(valImages)), axis = 1)
valECE = ECE(confidenceVal, predictedVal, valLabels, 15)

testAcc = model.evaluate(testImages, keras.utils.to_categorical(testLabels), verbose=0)[1]
predictedTest = np.argmax(model.predict(testImages), axis = 1)
confidenceTest = np.max(calibrate.predict(logitExtractor.predict(testImages)), axis = 1)
testECE = ECE(confidenceTest, predictedTest, testLabels, 15)

table = [['Measure', 'Acc', 'ECE'],
         ['Train', round(trainAcc, 2), round(trainECE, 2)],
         ['Val', round(valAcc, 2), round(valECE, 2)],
         ['Test', round(testAcc, 2), round(testECE, 2)]]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

reliabilityDiagram2(confidenceTest, predictedTest, testLabels, 15, 0)

### Search for high confidence errors

1. Calculate distances
2. Search for errors

#### Create features for derived feature space

- Consistent with previous literature

In [9]:
features = testImages.reshape(testImages.shape[0], testImages.shape[1] * testImages.shape[2] * testImages.shape[3])
x = StandardScaler().fit_transform(features)

pca = PCA(n_components=5)
principalComponents = np.round(pca.fit_transform(x), 4)
principalDf = pd.DataFrame(data = principalComponents, columns = ['P1', 'P2', 'P3', 'P4', 'P5'])

df = pd.DataFrame({'Prediction': predictedTest,
                   'Confidence': confidenceTest,
                   'True Label': testLabels,
                   'X1': principalDf.iloc[:,0].values,
                   'X2': principalDf.iloc[:,1].values,
                   'X3': principalDf.iloc[:,2].values,
                   'X4': principalDf.iloc[:,3].values,
                   'X5': principalDf.iloc[:,4].values,
                   'Misclassified': testLabels != predictedTest})

#### Calculate distance to turn image "adversarial"


In [ ]:
criticalClass = 0
gradientSteps = boundaryMSE(criticalClass, testImages, predictedTest, model, boundarySteps)
np.save('zapposBoundary1000', gradientSteps) 
boundDist = np.load('zapposBoundary1000.npy')
boundDist2 = np.log(boundDist)

#### Search for High Confidence Errors

In [ ]:
iterations = 1000
utilFeatures = df.iloc[:,3:8].values                             ## PCA features to calculate distances in derived feature space
##
## Subset to critical class
##
##  (boundDist already subsetted to critical class)
## 
utilFeatures = utilFeatures[predictedTest == criticalClass]
confidence = confidenceTest[predictedTest == criticalClass]
trueLabels = testLabels[predictedTest == criticalClass]
predictedLabels = predictedTest[predictedTest == criticalClass]

##
## Subset to high confidence
##
utilFeatures = utilFeatures[confidence > 0.65]
boundDist = boundDist2[confidence > 0.65]
trueLabels = trueLabels[confidence > 0.65]
predictedLabels = predictedLabels[confidence > 0.65]
confidence = confidence[confidence > 0.65]

##
## Hold SDR utility values
##
sdrRand = np.zeros(iterations*toSelect).reshape(iterations, toSelect)     ## Random search
sdrCon = np.zeros(iterations*toSelect).reshape(iterations, toSelect)      ## Low confidence search
sdrLow = np.zeros(iterations*toSelect).reshape(iterations, toSelect)      ## Adversarial distance search
sdrBW = np.zeros(iterations*toSelect).reshape(iterations, toSelect)       ## Bansal Weld search
sdrLak = np.zeros(iterations*toSelect).reshape(iterations, toSelect)      ## Lakaraju search

##
## Hold Bansal and Weld utility values
##
bwRand = np.zeros(iterations*toSelect).reshape(iterations, toSelect)     ## Random search
bwCon = np.zeros(iterations*toSelect).reshape(iterations, toSelect)      ## Low confidence search
bwLow = np.zeros(iterations*toSelect).reshape(iterations, toSelect)      ## Adversarial distance search
bwBW = np.zeros(iterations*toSelect).reshape(iterations, toSelect)       ## Bansal Weld search
bwLak = np.zeros(iterations*toSelect).reshape(iterations, toSelect)      ## Lakaraju search

##
## Hold Lakaraju utility values s
##
fixedRand = np.zeros(iterations*toSelect).reshape(iterations, toSelect)  ## Random search
fixedCon = np.zeros(iterations*toSelect).reshape(iterations, toSelect)   ## Low confidence search
fixedLow = np.zeros(iterations*toSelect).reshape(iterations, toSelect)   ## Adversarial distance search
fixedBW = np.zeros(iterations*toSelect).reshape(iterations, toSelect)    ## Bansal Weld search
fixedLak = np.zeros(iterations*toSelect).reshape(iterations, toSelect)   ## Lakaraju search

##
## Hold spread values
##
spreadRand = np.zeros(iterations*toSelect).reshape(iterations, toSelect) ## Random search
spreadCon = np.zeros(iterations*toSelect).reshape(iterations, toSelect)  ## Low confidence search
spreadLow = np.zeros(iterations*toSelect).reshape(iterations, toSelect)  ## Adversarial distance search
spreadBW = np.zeros(iterations*toSelect).reshape(iterations, toSelect)   ## Bansal Weld search
spreadLak = np.zeros(iterations*toSelect).reshape(iterations, toSelect)  ## Lakaraju search

##
## Hold confidence of sampled points
##
conRand = np.zeros(iterations*toSelect).reshape(iterations, toSelect)    ## Random search
conCon = np.zeros(iterations*toSelect).reshape(iterations, toSelect)     ## Low confidence search
conLow = np.zeros(iterations*toSelect).reshape(iterations, toSelect)     ## Adversarial distance search
conBW = np.zeros(iterations*toSelect).reshape(iterations, toSelect)      ## Bansal Weld search
conLak = np.zeros(iterations*toSelect).reshape(iterations, toSelect)     ## Lakaraju search

##
## Hold indicator if sampled point was correctly predicted
##
uuRand = np.zeros(iterations*toSelect).reshape(iterations, toSelect)    ## Random search
uuCon = np.zeros(iterations*toSelect).reshape(iterations, toSelect)     ## Low confidence search
uuLow = np.zeros(iterations*toSelect).reshape(iterations, toSelect)     ## Adversarial distance search
uuBW = np.zeros(iterations*toSelect).reshape(iterations, toSelect)      ## Bansal Weld search
uuLak = np.zeros(iterations*toSelect).reshape(iterations, toSelect)     ## Lakaraju search

##
## Get distance matrix
##
distances = distMatrix(principalComponents)
np.save('catDogDistance', distances) 

for i in range(iterations):
    print("\r" + str(i) + " of " + str(iterations), end="")
    ##
    ## Subset evaluation set and relevant information
    ##
    subset = random.sample(range(len(confidence)), sampleSize)                         ## Get random subset of evaluation dataset (changes initial conditions of search)
    utilFeaturesSubset = utilFeatures[subset]
    trueLabelsSubset = trueLabels[subset]
    predictedLabelsSubset = predictedLabels[subset]
    confidenceSubset = confidence[subset]
    advDistanceSubset = adversarialDistanceLOESS(boundDist[subset], confidenceSubset)  ## Calculate adversarial distance
    
    ##
    ## Perform searches
    ##
    rand = randSearch(len(confidenceSubset), toSelect)                                 ## Low confidence search
    con = lowConfidence(confidenceSubset, toSelect)                                    ## Low confidence search
    low = lowAdversarialDistance(advDistanceSubset, toSelect, confidenceSubset, 2.2)
    high = highAdversarialDistance(advDistanceSubset, toSelect)
   
    ##
    ## To run and evaluate code from Bansal Weld
    ##
    submod = SubmodUtilityModel(trueLabelsSubset, predictedLabelsSubset, confidenceSubset, var = 0.001)
    submod.setup(utilFeaturesSubset)
    
    fixed = UtilityModel(utilFeaturesSubset, trueLabelsSubset, predictedLabelsSubset, confidenceSubset, np.zeros(len(confidenceSubset)), gamma = 0.0)
    
    clusters = cluster('kmeans_both', utilFeaturesSubset, 6, confidenceSubset)
    k = len(np.unique(clusters))
    bwSearch, utilities = adap_greedy('conf', submod, toSelect, 'cluster', k, clusters)
    lakSearch, utilities = bandit_solution('uub', submod, k, clusters, toSelect)
    
    for j in range(0, toSelect):
        sdrRand[i,j] = SDR(confidenceSubset[rand[:j+1]], predictedLabelsSubset[rand[:j+1]], trueLabelsSubset[rand[:j+1]])
        sdrCon[i,j] = SDR(confidenceSubset[con[:j+1]], predictedLabelsSubset[con[:j+1]], trueLabelsSubset[con[:j+1]])
        sdrLow[i,j] = SDR(confidenceSubset[low[:j+1]], predictedLabelsSubset[low[:j+1]], trueLabelsSubset[low[:j+1]])
        sdrHigh[i,j] = SDR(confidenceSubset[high[:j+1]], predictedLabelsSubset[high[:j+1]], trueLabelsSubset[high[:j+1]])
        sdrBW[i,j] = SDR(confidenceSubset[bwSearch[:j+1]], predictedLabelsSubset[bwSearch[:j+1]], trueLabelsSubset[bwSearch[:j+1]])
        sdrLak[i,j] = SDR(confidenceSubset[lakSearch[:j+1]], predictedLabelsSubset[lakSearch[:j+1]], trueLabelsSubset[lakSearch[:j+1]])
        
        spreadRand[i,j] = spread(rand[:j+1], distances[np.ix_(subset,subset)])
        spreadCon[i,j] = spread(con[:j+1], distances[np.ix_(subset,subset)])
        spreadLow[i,j] = spread(low[:j+1], distances[np.ix_(subset,subset)])
        spreadHigh[i,j] = spread(high[:j+1], distances[np.ix_(subset,subset)])
        spreadBW[i,j] = spread(bwSearch[:j+1], distances[np.ix_(subset,subset)])
        spreadLak[i,j] = spread(lakSearch[:j+1], distances[np.ix_(subset,subset)])
      
        
        bwRand[i,j] = submod.get_utility(rand[:j+1])
        bwCon[i,j] = submod.get_utility(con[:j+1])
        bwLow[i,j] = submod.get_utility(low[:j+1])
        bwHigh[i,j] = submod.get_utility(high[:j+1])
        bwBW[i,j] = submod.get_utility(bwSearch[:j+1])
        bwLak[i,j] = submod.get_utility(lakSearch[:j+1])
        
        fixedRand[i,j] = fixed.get_utility(rand[:j+1])
        fixedCon[i,j] = fixed.get_utility(con[:j+1])
        fixedLow[i,j] = fixed.get_utility(low[:j+1])
        fixedHigh[i,j] = fixed.get_utility(high[:j+1])
        fixedBW[i,j] = fixed.get_utility(bwSearch[:j+1])
        fixedLak[i,j] = fixed.get_utility(lakSearch[:j+1])
        
        conRand[i,j] = confidenceSubset[rand[j]]
        conCon[i,j] = confidenceSubset[con[j]]
        conLow[i,j] = confidenceSubset[low[j]]
        conHigh[i,j] = confidenceSubset[high[j]]
        conBW[i,j] = confidenceSubset[bwSearch[j]]
        conLak[i,j] = confidenceSubset[lakSearch[j]]
        
        uuRand[i,j] = predictedLabelsSubset[rand[j]] != trueLabelsSubset[rand[j]]
        uuCon[i,j] = predictedLabelsSubset[con[j]] != trueLabelsSubset[con[j]]
        uuLow[i,j] = predictedLabelsSubset[low[j]] != trueLabelsSubset[low[j]]
        uuHigh[i,j] = predictedLabelsSubset[high[j]] != trueLabelsSubset[high[j]]
        uuBW[i,j] = predictedLabelsSubset[bwSearch[j]] != trueLabelsSubset[bwSearch[j]]
        uuLak[i,j] = predictedLabelsSubset[lakSearch[j]] != trueLabelsSubset[lakSearch[j]]

In [ ]:
import itertools
df1 = pd.DataFrame({'phi': 'BansalWeld',
                    'iteration': [i for i, j in itertools.product(range(sdrBW.shape[0]), range(sdrBW.shape[1]))],
                    'b': [j for i, j in itertools.product(range(sdrBW.shape[0]), range(sdrBW.shape[1]))],
                    'SDR': [sdrBW[i,j] for i, j in itertools.product(range(sdrBW.shape[0]), range(sdrBW.shape[1]))],
                    'spread': [spreadBW[i,j] for i, j in itertools.product(range(sdrBW.shape[0]), range(sdrBW.shape[1]))],
                    'BW':[bwBW[i,j] for i, j in itertools.product(range(sdrBW.shape[0]), range(sdrBW.shape[1]))],
                    'Fixed':[fixedBW[i,j] for i, j in itertools.product(range(sdrBW.shape[0]), range(sdrBW.shape[1]))],
                    'Confidence':[conBW[i,j] for i, j in itertools.product(range(conBW.shape[0]), range(conBW.shape[1]))],
                    'UnknownUnknown':[uuBW[i,j] for i, j in itertools.product(range(uuBW.shape[0]), range(uuBW.shape[1]))]})

df2 = pd.DataFrame({'phi': 'Lakkaraju',
                    'iteration': [i for i, j in itertools.product(range(sdrLak.shape[0]), range(sdrLak.shape[1]))],
                    'b': [j for i, j in itertools.product(range(sdrLak.shape[0]), range(sdrLak.shape[1]))],
                    'SDR': [sdrLak[i,j] for i, j in itertools.product(range(sdrLak.shape[0]), range(sdrLak.shape[1]))],
                    'spread': [spreadLak[i,j] for i, j in itertools.product(range(sdrLak.shape[0]), range(sdrLak.shape[1]))],
                    'BW':[bwLak[i,j] for i, j in itertools.product(range(sdrLak.shape[0]), range(sdrLak.shape[1]))],
                    'Fixed':[fixedLak[i,j] for i, j in itertools.product(range(sdrLak.shape[0]), range(sdrLak.shape[1]))],
                    'Confidence':[conLak[i,j] for i, j in itertools.product(range(conLak.shape[0]), range(conLak.shape[1]))],
                    'UnknownUnknown':[uuLak[i,j] for i, j in itertools.product(range(uuLak.shape[0]), range(uuLak.shape[1]))]})

df3 = pd.DataFrame({'phi': 'lowConfidence',
                    'iteration': [i for i, j in itertools.product(range(sdrCon.shape[0]), range(sdrCon.shape[1]))],
                    'b': [j for i, j in itertools.product(range(sdrCon.shape[0]), range(sdrCon.shape[1]))],
                    'SDR': [sdrCon[i,j] for i, j in itertools.product(range(sdrCon.shape[0]), range(sdrCon.shape[1]))],
                    'spread': [spreadCon[i,j] for i, j in itertools.product(range(sdrCon.shape[0]), range(sdrCon.shape[1]))],
                    'BW':[bwCon[i,j] for i, j in itertools.product(range(sdrCon.shape[0]), range(sdrCon.shape[1]))],
                    'Fixed':[fixedCon[i,j] for i, j in itertools.product(range(sdrCon.shape[0]), range(sdrCon.shape[1]))],
                    'Confidence':[conCon[i,j] for i, j in itertools.product(range(conCon.shape[0]), range(conCon.shape[1]))],
                    'UnknownUnknown':[uuCon[i,j] for i, j in itertools.product(range(uuCon.shape[0]), range(uuCon.shape[1]))]})

df4 = pd.DataFrame({'phi': 'random',
                    'iteration': [i for i, j in itertools.product(range(sdrRand.shape[0]), range(sdrRand.shape[1]))],
                    'b': [j for i, j in itertools.product(range(sdrRand.shape[0]), range(sdrRand.shape[1]))],
                    'SDR': [sdrRand[i,j] for i, j in itertools.product(range(sdrRand.shape[0]), range(sdrRand.shape[1]))],
                    'spread': [spreadRand[i,j] for i, j in itertools.product(range(sdrRand.shape[0]), range(sdrRand.shape[1]))],
                    'BW':[bwRand[i,j] for i, j in itertools.product(range(sdrRand.shape[0]), range(sdrRand.shape[1]))],
                    'Fixed':[fixedRand[i,j] for i, j in itertools.product(range(sdrRand.shape[0]), range(sdrRand.shape[1]))],
                    'Confidence':[conRand[i,j] for i, j in itertools.product(range(conRand.shape[0]), range(conRand.shape[1]))],
                    'UnknownUnknown':[uuRand[i,j] for i, j in itertools.product(range(uuRand.shape[0]), range(uuRand.shape[1]))]})


df5 = pd.DataFrame({'phi': 'adversarial',
                    'iteration': [i for i, j in itertools.product(range(sdrLow.shape[0]), range(sdrLow.shape[1]))],
                    'b': [j for i, j in itertools.product(range(sdrLow.shape[0]), range(sdrLow.shape[1]))],
                    'SDR': [sdrLow[i,j] for i, j in itertools.product(range(sdrLow.shape[0]), range(sdrLow.shape[1]))],
                    'spread': [spreadLow[i,j] for i, j in itertools.product(range(sdrLow.shape[0]), range(sdrLow.shape[1]))],
                    'BW':[bwLow[i,j] for i, j in itertools.product(range(sdrLow.shape[0]), range(sdrLow.shape[1]))],
                    'Fixed':[fixedLow[i,j] for i, j in itertools.product(range(sdrLow.shape[0]), range(sdrLow.shape[1]))],
                    'Confidence':[conLow[i,j] for i, j in itertools.product(range(conLow.shape[0]), range(conLow.shape[1]))],
                    'UnknownUnknown':[uuLow[i,j] for i, j in itertools.product(range(uuLow.shape[0]), range(uuLow.shape[1]))]})

toSave = pd.concat([df1, df2, df3, df4, df5])
toSave.to_csv('resultsZappos.csv', sep=',')